In [1]:
from selenium import webdriver
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import math
import statistics
import resources

## Scrapping the data
- Entering url and stracting data as a Pandas df
- After scrapping the data, df will be stored as CSV file
- If data has been already scrapped, data is stracted from CSV

In [2]:
# Check if exist a .csv with filename equals to TODAY DATETIME
if True:
#     NON_DESIRED_TEAMS = ['Team Durant','Team LeBron']
    # Getting data
    df = resources.data_scraper.scrape()
    # Casting columns
    df[df.columns[2]] = df[df.columns[2]].astype(np.int64)
    df[df.columns[3]] = df[df.columns[3]].astype(np.int64)
    # Adding total_score column
    df["TotalScore"] = df[df.columns[2]] + df[df.columns[3]]
    # Store CSV
    df.to_csv('data/30-04.csv', encoding='utf-8', index=False)
else:
    df = pd.read_csv('data/30-04.csv')

Discarding following match Washington Wizards vs San Antonio Spurs
Discarding following match New York Knicks vs Atlanta Hawks
Discarding following match Denver Nuggets vs Memphis Grizzlies
Discarding following match Milwaukee Bucks vs Phoenix Suns
Discarding following match New York Knicks vs New Orleans Pelicans
Discarding following match Los Angeles Lakers vs Utah Jazz
Discarding following match Washington Wizards vs New Orleans Pelicans
Discarding following match Boston Celtics vs Minnesota Timberwolves
Discarding following match New York Knicks vs Memphis Grizzlies
Discarding following match Phoenix Suns vs Utah Jazz
Discarding following match San Antonio Spurs vs Indiana Pacers
Discarding following match San Antonio Spurs vs Atlanta Hawks
Discarding following match New Orleans Pelicans vs Orlando Magic
Discarding following match Charlotte Hornets vs Phoenix Suns
Discarding following match Memphis Grizzlies vs Boston Celtics
Discarding following match New York Knicks vs Philadelph

In [3]:
# NON_DESIRED_TEAMS = ['Team Durant','Team LeBron']
# df = df[df[df.columns[0]] != (NON_DESIRED_TEAMS[0] or NON_DESIRED_TEAMS[1])]
# df = df[df[df.columns[1]] != (NON_DESIRED_TEAMS[0] or NON_DESIRED_TEAMS[1])]
len(df) # 833 for today

937

#### Heatmap

In [4]:
# resources.graph.heatmap(df, f_size=10, cmap="afmhot_r",vmin=150, vmax=250)

### Frecuency histogram
- To know how many divisions or 'bins' should be done in the histogram, we must use Sturges law

$$ c = 1 + \dfrac{log(M)}{log(2)}$$

In [5]:
# resources.graph.frecuency_histogram(df)


#### Boxplot

In [6]:
# plt.boxplot(df['TotalScore'])
# plt.title("Total score distribution")

#### Normal dfistribution

In [7]:
# resources.graph.normal_distribution(df, 'Atlanta Hawks')

## Data analysis
- Analysing data
- Store results in a new df

In [8]:
stats_df = resources.nba_stats.stats(df)

## Manually studying data

In [9]:
conversion_dict = {
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BKN': 'Brooklyn Nets',
    'CHA': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GS': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NO': 'New Orleans Pelicans',
    'NY': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHX': 'Phoenix Suns',
    'PHI': 'Philadelphia 76ers',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SA': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

In [10]:
future_games = [
    ['ATL', 'CHA'],
    ['BOS', 'DEN'],
    ['MIL', 'ORL'],
    ['NO', 'CLE'],
    ['CHI', 'MIN'],
    ['IND', 'MEM'],
    ['SA', 'DAL'],
    ['TOR', 'NY'],
    ['DET', 'LAC'],
    ['MIA', 'POR'],    
]

games_converted = list()
               
for game in future_games:
    home_team = conversion_dict[game[0]]
    away_team = conversion_dict[game[1]]
    game_ = [home_team, away_team]
    games_converted.append(game_)
    
for game in games_converted:
    df_ = stats_df.query("team == '{0}' or team == '{1}'".format(game[0], game[1]))
#     print(df_)
    print(df_.iloc[:,[0,1,4,5]])
    print("\n\n")
        

                 team        mean  factorDown  factorUp
17  Charlotte Hornets  221.359375    4.687500  6.250000
20      Atlanta Hawks  222.476190    7.936508  6.349206



              team        mean  factorDown  factorUp
0   Denver Nuggets  222.933333    6.666667  6.666667
10  Boston Celtics  220.145161   16.129032  6.451613



               team        mean  factorDown   factorUp
19    Orlando Magic  217.390625    9.375000   1.562500
25  Milwaukee Bucks  232.619048    1.587302  28.571429



                    team        mean  factorDown   factorUp
13   Cleveland Cavaliers  212.809524   23.809524   3.174603
18  New Orleans Pelicans  229.016667    8.333333  18.333333



                      team        mean  factorDown  factorUp
3   Minnesota Timberwolves  225.870968    6.451613  8.064516
26           Chicago Bulls  221.645161    6.451613  6.451613



                team        mean  factorDown   factorUp
5     Indiana Pacers  226.847458    8.474576  16.949153
8  Memphis Grizzli

## Training model

In [11]:
def normalize(x, min_, max_):
    x = float(x)
    if x >= max_:
        return 1
    elif x <= min_:
        return 0
    return (x - min_) / (float(max_) - min_)

def denormalize(z, min_, max_):
    return z * (max_ - min_) + min_
    

def prepare_data(df):
    down, mean, up = df['factorDown'], df['mean'], df['factorUp']
    mean = normalize(mean, 165, 250)
    down = float(down)/100
    up = float(up)/100
    return down, mean, up
    

for index, row in df.iterrows():
    home, away, score = row['HomeTeam'], row['AwayTeam'], row['TotalScore']
    df_h = stats_df.query("team == '{0}'".format(home))
    df_a = stats_df.query("team == '{0}'".format(away))

    h_down, h_mean, h_up = prepare_data(df_h)
    a_down, a_mean, a_up = prepare_data(df_a)
    
    # Input for ML model
    X = np.array([h_down, h_mean, h_up, a_up, a_mean, a_up])
    # Expected result
    y = normalize(score, 165, 250)   
    
    print('Training model with input:\nX: {}\nExpected output:\ny: {}\n\n'.format(X, y))
    
    if index>=5:
        break

Training model with input:
X: [0.06666667 0.68156863 0.06666667 0.10606061 0.68003565 0.10606061]
Expected output:
y: 0.788235294117647


Training model with input:
X: [0.07936508 0.63902894 0.04761905 0.18333333 0.75313725 0.18333333]
Expected output:
y: 0.4588235294117647


Training model with input:
X: [0.13636364 0.67825312 0.09090909 0.28571429 0.79551821 0.28571429]
Expected output:
y: 1


Training model with input:
X: [0.06451613 0.71612903 0.08064516 0.13846154 0.71927602 0.13846154]
Expected output:
y: 0.8823529411764706


Training model with input:
X: [0.15873016 0.58804855 0.06349206 0.11111111 0.66386555 0.11111111]
Expected output:
y: 0.6470588235294118


Training model with input:
X: [0.08474576 0.72761715 0.16949153 0.2295082  0.77994214 0.2295082 ]
Expected output:
y: 0.9176470588235294


